In [1]:
import requests as r
from bs4 import BeautifulSoup as bs
import json 
from queue import Queue
import threading
import re
import time
import random
import os

In [359]:
#pre_url_str = 'http://www.ipeen.com.tw/shop/1055186-%E6%BF%83%E5%92%96%E5%95%A1-PEI-CAFE-%E5%90%8E%E9%87%8C'
#pre_url_str = 'http://www.ipeen.com.tw/shop/1071950-%E7%B4%85%E8%91%89%E8%9B%8B%E7%B3%95'
pre_url_str = 'http://www.ipeen.com.tw/shop/33714-%E5%92%96%E5%95%A1%E5%BC%84-%E5%BF%A0%E5%AD%9D%E5%BA%97'

## 需要執行

In [360]:
restaurant_id = pre_url_str.split('/')[-1].split('-')[0]

#測試將軍漁港，有兩個營業時間，看看怎麼清理
#1.<td>  今日 <span>未設定</span> </td> 2.<td>PM17:00~AM02:00</td>
#restaurant_id = 145359
#測試濃咖啡 PEI CAFE(后里)，沒有營業時間
#restaurant_id = 1055186

restaurant_information_url = 'http://www.ipeen.com.tw/touch/shop.php?id={}'.format(restaurant_id)

In [361]:
restaurant_id

'33714'

In [362]:
restaurant_information_url

'http://www.ipeen.com.tw/touch/shop.php?id=33714'

## 需要執行

In [363]:
phoneStartTime = time.time()
#test_restaurant_information_url = 'http://www.ipeen.com.tw/touch/shop.php?id=1055186'
#test_restaurant_information_url = 'http://www.ipeen.com.tw/touch/shop.php?id=1071950'
#test_restaurant_information_url = 'http://www.ipeen.com.tw/touch/shop.php?id=33714'
header1 = {'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
          'Accept-Encoding': 'gzip, deflate, sdch',
          'Accept-Language': 'zh-TW,zh;q=0.8,en-US;q=0.6,en;q=0.4',
          'Cache-Control': 'no-cache',
          'Connection': 'keep-alive',
          'Host': 'www.ipeen.com.tw',
          'Pragma': 'no-cache',
          'DNT': '1',
          'Upgrade-Insecure-Requests': '1',
          'User-Agent': 'Mozilla/5.0 (iPhone; CPU iPhone OS 9_1 like Mac OS X) AppleWebKit/601.1.46 (KHTML, like Gecko) \
          Version/9.0 Mobile/13B143 Safari/601.1'
          }
res_restaurant_information = r.get(restaurant_information_url, headers=header1)
print(res_restaurant_information.status_code)
soup_restaurant_information = bs(res_restaurant_information.text, 'lxml')
phoneEndTime = time.time()
print(phoneEndTime - phoneStartTime)

200
0.44402527809143066


# 店名

## 需要執行

In [364]:
restaurant_name = soup_restaurant_information.find('span', {'itemprop': 'name'}).text
restaurant_name

'咖啡弄'

# 餐廳地址與googleMapUrl

## 需要執行

In [365]:
google_map_url = soup_restaurant_information.find('a', {'href': re.compile('(http:\/\/maps\.google\.com\/maps\?q=)')})['href']
print(google_map_url)

def addressRepl(matchobj):
    return matchobj.group(2)
restaurant_address = re.sub('(http:\/\/maps\.google\.com\/maps\?q=)(.+$)',addressRepl ,google_map_url)
print(restaurant_address)

http://maps.google.com/maps?q=台北市台北市大安區忠孝東路四段101巷45號
台北市台北市大安區忠孝東路四段101巷45號


# 電話

## 需要執行

In [266]:
phone_pattarn = '(tel:)?(\(?\d{2}\)?)?(-|\s+)?(\d{3}|\d{4})(-|\s+)?(\d{4})'

## 需要執行

In [366]:
phone_number = soup_restaurant_information.find('span', {'itemprop': 'telephone'}).text
if re.match(phone_pattarn, phone_number):
    print(phone_number)

02-2711-1912


# 標籤ul底下 四大標籤li出現順序為1.綜合評分 2.我的評分 3.(平均消費) > 餐廳類別

## 需要執行

In [367]:
first_li_under_ul = soup_restaurant_information.find('li', {'itemprop': 'aggregateRating'})

# 綜合評分

## 需要執行

In [368]:
aggregateRating_int = int(first_li_under_ul.find('span', {'itemprop': 'ratingValue'}).text)
if aggregateRating_int != 0:
    print(aggregateRating_int)
else:
    print('無有效綜合評分')

45


# 平均消費&餐廳類別

## 需要執行

In [369]:
third_li_under_ul = first_li_under_ul.find_next_sibling().find_next_sibling()
if re.match('平均消費', third_li_under_ul.text):
    avg_price = int(third_li_under_ul.text.split('：')[1])
    print(avg_price)
    category = third_li_under_ul.find_next_sibling().text
    print(category)
else:
    print('沒有均消')
    category = third_li_under_ul.text
    print(category)

269
甜點專賣


# 營業時間&官方網站&推薦菜&分類標籤

In [373]:
soup_restaurant_information.table

<table>
<tr>
<th>商家名稱</th>
<td>咖啡弄(忠孝店)                </td>
</tr>
<tr>
<th>電話</th>
<td>
<a class="ga_tracking" data-action="details_phone" data-category="MWEB_shop" data-label="詳細資訊-電話" href="tel:02-2711-1912">02-2711-1912</a> </td>
</tr>
<tr>
<th>地址</th>
<td>台北市大安區忠孝東路四段101巷45號            
                                            <a href="../touch/traffic.php?id=33714"><p>(詳細交通資訊)</p></a>
</td>
</tr>
<tr>
<th>捷運資訊</th>
<td>近台北市捷運 - 板南線 - 忠孝敦化站 (步行約6分鐘)<br/></td>
</tr>
<tr>
<th>公休日</th>
<td>無</td>
</tr>
<tr>
<th>營業時間</th>
<td>
                    今日 <span>12:00~23:00</span> <a class="business-hour-show-all" href="#">顯示全部</a>
<div class="business-hour-pannel">
<div class="business-hour-detail hide">
<table>
<tr class="today">
<th>週一</th>
<td>
                                                                                      12:00~23:00                                                      </td>
</tr>
<tr class="">
<th>週二</th>
<td>
                                                  

## 需要執行

In [370]:
all_th_in_table = soup_restaurant_information.table.findAll('th')

In [406]:
all_th_in_table

[<th>商家名稱</th>,
 <th>電話</th>,
 <th>地址</th>,
 <th>捷運資訊</th>,
 <th>公休日</th>,
 <th>營業時間</th>,
 <th>週一</th>,
 <th>週二</th>,
 <th>週三</th>,
 <th>週四</th>,
 <th>週五</th>,
 <th>週六</th>,
 <th>週日</th>,
 <th>補充說明</th>,
 <th>席       位</th>,
 <th>可否刷卡</th>,
 <th>官方網站</th>,
 <th>推 薦 菜</th>,
 <th>分類標籤</th>]

# 營業時間

In [275]:
all_th_in_table[5].find_next_sibling()

<td>PM17:00~AM02:00</td>

### 時間的正規表達式

## 需要執行

In [295]:
time_pattern  = '(?:[Aa][Mm]|[Pp][Mm])?((?:[01][0-9]|2[0-3]?):([0-5][0-9]))'

In [ ]:
#  match
# 12:00
# AM23:00
# Pm11:59
# 23:59
# 02:00

#  !match
# 35:22
# 4:32
# 13:88

## 需要執行

In [296]:
timeInterval_pattern = '{}~{}'.format(time_pattern,time_pattern)

In [ ]:
#  match
# am11:30~Pm20:30
# PM17:00~AM02:00
# 12:00~23:00
# 11:59~23:59

#  !match
# 35:22~02:00
# 02:60~23:00

In [304]:
timeInterval_pattern

'(?:[Aa][Mm]|[Pp][Mm])?((?:[01][0-9]|2[0-3]?):([0-5][0-9]))~(?:[Aa][Mm]|[Pp][Mm])?((?:[01][0-9]|2[0-3]?):([0-5][0-9]))'

In [328]:
re.match(timeInterval_pattern, 'pm12:00~23:00  ')

<_sre.SRE_Match object; span=(0, 13), match='pm12:00~23:00'>

In [ ]:
# 定義Repl程式

## 需要執行

In [323]:
def timeIntervalRepl(matchobj):
    return matchobj.group(1)+ '~' + matchobj.group(3)

In [ ]:
# 現在可以把所有無意義的ampm拿掉了，反正好像都是24小時制

In [329]:
open_hours = re.sub(timeInterval_pattern,timeIntervalRepl , 'am12:00~pm23:00')
open_hours

'12:00~23:00'

In [330]:
# 注意，re.match資料檢察不能拿掉，不然re.sub()可能會回傳奇怪的東西。以下舉例
open_hours = re.sub(timeInterval_pattern,timeIntervalRepl , 'sm12:00~pm23:00')
open_hours

'sm12:00~23:00'

## 需要執行

In [405]:

for th in all_th_in_table:
    if th.text=='營業時間':
        #今日時間有a標籤，代表多個營業時間
        if th.find_next_sibling().a:
            print('多個時間')
            th_open_hours_list = th.find_next_sibling().findAll('th')
            #在所有th標籤裡可以找到星期幾的標籤，然後用th的下一個平輩標籤find_next_sibling()找到當天的開店時間
            #然後檢查時間格式是否符合標準
            for th in th_open_hours_list:
                #print(th)
                if th.text=='週一':
                    Monday = th.text
                    print(Monday)
                    pre_opne_hours_MON = th.find_next_sibling().text.strip()
                    if re.match(timeInterval_pattern, pre_opne_hours_MON):
                        print('有正確時間')
                        opne_hours_MON = re.sub(timeInterval_pattern,timeIntervalRepl , pre_opne_hours_MON)
                        print(opne_hours_MON)
                    else:
                        opne_hours_MON = None
                elif th.text=='週二':
                    Tuesday = th.text
                    print(Tuesday)
                    pre_opne_hours_TUE = th.find_next_sibling().text.strip()
                    if re.match(timeInterval_pattern, pre_opne_hours_TUE):
                        print('有正確時間')
                        opne_hours_TUE = re.sub(timeInterval_pattern,timeIntervalRepl , pre_opne_hours_TUE)
                        print(opne_hours_TUE)
                    else:
                        opne_hours_TUE = None
                elif th.text=='週三':
                    Wednesday = th.text
                    print(Wednesday)
                    pre_opne_hours_WED = th.find_next_sibling().text.strip()
                    if re.match(timeInterval_pattern, pre_opne_hours_WED):
                        print('有正確時間')
                        opne_hours_WED = re.sub(timeInterval_pattern,timeIntervalRepl , pre_opne_hours_WED)
                        print(opne_hours_WED)
                    else:
                        opne_hours_WED = None
                elif th.text=='週四':
                    Thursday = th.text
                    print(Thursday)
                    pre_opne_hours_THU = th.find_next_sibling().text.strip()
                    if re.match(timeInterval_pattern, pre_opne_hours_THU):
                        print('有正確時間')
                        opne_hours_THU = re.sub(timeInterval_pattern,timeIntervalRepl , pre_opne_hours_THU)
                        print(opne_hours_THU)
                    else:
                        opne_hours_THU = None
                elif th.text=='週五':
                    Friday = th.text
                    print(Friday)
                    pre_opne_hours_FRI = th.find_next_sibling().text.strip()
                    if re.match(timeInterval_pattern, pre_opne_hours_FRI):
                        print('有正確時間')
                        opne_hours_FRI = re.sub(timeInterval_pattern,timeIntervalRepl , pre_opne_hours_FRI)
                        print(opne_hours_FRI)
                    else:
                        opne_hours_FRI = None
                elif th.text=='週六':
                    Saturday = th.text
                    print(Saturday)
                    pre_opne_hours_SAT = th.find_next_sibling().text.strip()
                    if re.match(timeInterval_pattern, pre_opne_hours_SAT):
                        print('有正確時間')
                        opne_hours_SAT = re.sub(timeInterval_pattern,timeIntervalRepl , pre_opne_hours_SAT)
                        print(opne_hours_SAT)
                    else:
                        opne_hours_SAT = None
                    
                elif th.text=='週日':
                    Sunday = th.text
                    print(Sunday)
                    pre_opne_hours_SUN = th.find_next_sibling().text.strip()
                    if re.match(timeInterval_pattern, pre_opne_hours_SUN):
                        print('有正確時間')
                        opne_hours_SUN = re.sub(timeInterval_pattern,timeIntervalRepl , pre_opne_hours_SUN)
                        print(opne_hours_SUN)
                    else:
                        opne_hours_SUN = None
                else:
                    pass
                print('----------')
            print(opne_hours_MON, opne_hours_TUE,opne_hours_WED,opne_hours_THU,opne_hours_FRI,opne_hours_SAT,opne_hours_SUN)
        #今日時間如果沒有a標籤，代表沒有多個營業時間或是沒有設定營業時間
        else:
            print('一個時間或未設定營業時間')
            pre_open_hours = th.find_next_sibling().text
            print(pre_open_hours)
            if re.match(timeInterval_pattern, pre_open_hours):
                print('有正確時間')
                open_hours = re.sub(timeInterval_pattern,timeIntervalRepl , pre_open_hours)
                print(open_hours)
            else:
                print('未設定營業時間')
            #break

多個時間
週一
有正確時間
12:00~23:00
----------
週二
有正確時間
12:00~23:00
----------
週三
有正確時間
12:00~23:00
----------
週四
有正確時間
12:00~23:00
----------
週五
有正確時間
12:00~23:00
----------
週六
有正確時間
12:00~23:00
----------
週日
有正確時間
12:00~23:00
----------
12:00~23:00 12:00~23:00 12:00~23:00 12:00~23:00 12:00~23:00 12:00~23:00 12:00~23:00


In [407]:
all_th_in_table

[<th>商家名稱</th>,
 <th>電話</th>,
 <th>地址</th>,
 <th>捷運資訊</th>,
 <th>公休日</th>,
 <th>營業時間</th>,
 <th>週一</th>,
 <th>週二</th>,
 <th>週三</th>,
 <th>週四</th>,
 <th>週五</th>,
 <th>週六</th>,
 <th>週日</th>,
 <th>補充說明</th>,
 <th>席       位</th>,
 <th>可否刷卡</th>,
 <th>官方網站</th>,
 <th>推 薦 菜</th>,
 <th>分類標籤</th>]

# 官方網站

# 推薦菜

# 分類標籤

In [418]:
for th in all_th_in_table:
    if th.text=='官方網站':
        print(th)
        if th.find_next_sibling().a:
            print('有官方網站超連結')            
            officialWebsite_url = th.find_next_sibling().a['href']
            print(officialWebsite_url)
    if th.text=='推 薦 菜':
        print(th)
        if th.find_next_sibling().a:
            print('有推薦菜')
            a_detailsRecommend_list = th.find_next_sibling().select('a')
            #print(a_detailsRecommend_list)
            for a in a_detailsRecommend_list:
                recommendation = a.text
                print(recommendation)
            
    if th.text=='分類標籤':
        print(th)
        if th.find_next_sibling().a:
            print('有分類標籤')
            a_Tags_list = th.find_next_sibling().select('a')
            #print(a_Tags_list)
            for a in a_Tags_list:
                Tag = a.text
                print(Tag)

<th>官方網站</th>
有官方網站超連結
http://www.coffee-alley.com.tw
<th>推 薦 菜</th>
有推薦菜
草莓冰淇淋鬆餅(94)
鬆餅(32)
總匯三明治(27)
BBQ豬排三明治(17)
鮮果茶(17)
<th>分類標籤</th>
有分類標籤
下午茶(152)
鬆餅(142)
咖啡弄(95)
朋友聚會(91)
東區(78)


# 食記分享

## 需要執行

In [372]:
#判斷是否有食記分享
if soup_restaurant_information.find('div', {'id': 'comment'}):    
    print('有分享')    
    #取得總共有幾篇
    pre_comment_number_str = soup_restaurant_information.find('div', {'id': 'comment'}).h3.text
    def numberRepl(matchobj):
        return matchobj.group(2)
    comment_number = int(re.sub('(會員分享文\(共)(\d+)( 篇\))',numberRepl ,pre_comment_number_str))
    print(comment_number)
    
    #依照篇數，計算一共有幾頁
    if comment_number <= 5:
        int_page = 1
        print(int_page)
    elif comment_number %5 == 0:
        int_page=int(comment_number/5)
        print(int_page)
    else:
        int_page=int(comment_number/5)+1
        print(int_page)
    
    #產生分享文列表的全部頁數的url
    for i in range(1,int_page+1):
        comment_page_url = 'http://www.ipeen.com.tw/touch/cmmList.php?p={}&id={}'.format(i, restaurant_id)
        print(comment_page_url)
        #restaurant_share(comment_page_url)
else:
    print('尚無分享')

有分享
206
42
http://www.ipeen.com.tw/touch/cmmList.php?p=1&id=33714
http://www.ipeen.com.tw/touch/cmmList.php?p=2&id=33714
http://www.ipeen.com.tw/touch/cmmList.php?p=3&id=33714
http://www.ipeen.com.tw/touch/cmmList.php?p=4&id=33714
http://www.ipeen.com.tw/touch/cmmList.php?p=5&id=33714
http://www.ipeen.com.tw/touch/cmmList.php?p=6&id=33714
http://www.ipeen.com.tw/touch/cmmList.php?p=7&id=33714
http://www.ipeen.com.tw/touch/cmmList.php?p=8&id=33714
http://www.ipeen.com.tw/touch/cmmList.php?p=9&id=33714
http://www.ipeen.com.tw/touch/cmmList.php?p=10&id=33714
http://www.ipeen.com.tw/touch/cmmList.php?p=11&id=33714
http://www.ipeen.com.tw/touch/cmmList.php?p=12&id=33714
http://www.ipeen.com.tw/touch/cmmList.php?p=13&id=33714
http://www.ipeen.com.tw/touch/cmmList.php?p=14&id=33714
http://www.ipeen.com.tw/touch/cmmList.php?p=15&id=33714
http://www.ipeen.com.tw/touch/cmmList.php?p=16&id=33714
http://www.ipeen.com.tw/touch/cmmList.php?p=17&id=33714
http://www.ipeen.com.tw/touch/cmmList.php?p=18

# 餐廳菜單連結

## 需要執行

In [435]:
if soup_restaurant_information.find('a', {'href': re.compile('(\.\./touch/menu.php\?id=)')}):
    print('有餐廳菜單')
    pre_menu_url = soup_restaurant_information.find('a', {'href': re.compile('(\.\./touch/menu.php\?id=)')})['href']
    replace_pattern = '..'
    menu_url = pre_menu_url.replace(replace_pattern, 'http://www.ipeen.com.tw')
    print(menu_url)

有餐廳菜單
http://www.ipeen.com.tw/touch/menu.php?id=33714


# 切換頁面，進到餐廳菜單

## 需要執行

In [454]:
try:
    res_menu = r.get(menu_url, headers=header1)
except HTTPError:
    print('HTTPError')
    #return None
print(res_menu.status_code)
res_menu.encoding="utf-8"
soup_menu = bs(res_menu.text, 'lxml')

200


## 需要執行

In [441]:
menuClassification_list = soup_menu.findAll('ul', {'class': 'group'})

## 需要執行

In [451]:
for group in menuClassification_list:
    #print(group)
    #菜單種類標題
    classificationTitle = group.h6.text
    print(classificationTitle)
    
    food_in_group_list = group.select('ul > li')
    #單品
    for food in food_in_group_list:
        #名稱
        foodName_str = food.h6.text          
        print(foodName_str)
        #價格
        pre_foodPrice_str = food.span.text
        foodPrice_str = pre_foodPrice_str.split('.')[1]
        int(foodPrice_str)
        print(foodPrice_str)
    print('-----------------')

手工熱咖啡
藍山咖啡
160
歐蕾咖啡
150
巴西山多士咖啡
120
曼特寧咖啡
120
炭燒咖啡
120
綜合咖啡
120
特調手工咖啡
130
維也納咖啡
130
香草咖啡
140
-----------------
義式咖啡
義式濃縮咖啡 single
90
義式濃縮咖啡 duble
100
康寶藍
100
美式淡咖啡
110
低咖啡因咖啡
120
卡布奇諾咖啡
120
拿鐵咖啡
120
美式摩卡咖啡
130
焦糖瑪奇朵
130
提拉米蘇咖啡
130
-----------------
餐點
手工蛋糕 布朗尼
100
手工捲餅
110
原味鬆餅
100
煉乳花生醬鬆餅
120
草莓冰淇淋鬆餅
150
紅豆抹茶冰淇淋鬆餅
150
鮮果冰淇淋鬆餅
150
香蕉巧克力冰淇淋鬆餅
150
蔬菜鮪魚鬆餅
150
燻雞土司披薩
130
燻牛肉三明治
150
總匯三明治
140
ＢＢＱ豬排三明治
150
燻雞肉三明治
130
芥末培根起士三明治
150
鮪魚三明治
130
帕瑪森凱薩沙拉
120
蛋沙拉三明治
120
-----------------
熱茶飲
熱可可亞
130
熱抹茶歐蕾
120
法式奶茶
130
紅茶歐蕾（原味/香橙）
150
鮮果茶
160
柚桔茶
150
伯爵紅茶
140
錫蘭紅茶
140
高山烏龍茶
140
俄羅斯果醬茶
150
桂圓紅棗茶
150
玫瑰花草茶
140
絲路花草茶
140
菩提子花草茶
140
菊花茶
140
茉莉花草茶
140
蘋果檸香茶
140
冷薄荷花草茶
140
蜂蜜生薑茶
140
-----------------
冰咖啡
冰黑咖啡
120
特弄冰咖啡
130
冰滴咖啡
140
冰香草咖啡
140
冰維也納咖啡
130
冰牛奶咖啡
120
冰拿鐵咖啡
120
冰美式摩卡咖啡
130
-----------------
冰淇淋甜點
自由之丘
100
御茶之水
150
紅豆抹茶冰淇淋
150
鮮草莓冰淇淋百匯
160
鮮果冰淇淋百匯
160
-----------------
冰茶飲
冰柚桔茶
150
冰鮮果茶
160
冰弄紅茶
100
檸檬凍茶
120
黃檸檬蘇打
120
鮮榨季節果汁
130
皇家冰奶茶
120
冰相思歐蕾
130
冰抹茶歐蕾
130
冰可可亞
130
冰淇淋紅茶
130
咖啡冰沙
130
抹茶牛奶冰沙


# 切換頁面，進到食記分享文列表

## 需要執行

In [110]:
def restaurant_share(comment_page_url):
    
    try:
        res_comment_page = r.get(comment_page_url, headers=header1)
    except HTTPError:
            return None
    print(res_comment_page.status_code)
    res_comment_page.encoding="utf-8"
    soup_comment_page = bs(res_comment_page.text, 'lxml')
    
    
    pre_comment_url_list = soup_comment_page.findAll('ol')
    
    for pre_comment_url in pre_comment_url_list:
        this_pre_comment_url = pre_comment_url.parent['href']
        print(this_pre_comment_url)
        comment_id = this_pre_comment_url.split('=')[-1]
        print(comment_id)
        comment_url = 'http://www.ipeen.com.tw{}'.format(this_pre_comment_url)
        print(comment_url)
        comment_title = pre_comment_url.li.text
        print(comment_title)